# Exercise 1 - Logistic Regression

In [1]:
import tensorflow as tf

## Objectives

In this exercise we will implement the following functions:
* `softmax`: computes the softmax (normalised exponential function) of a input tensor;
* `cross_entropy`: calculates the cross-entropy loss between one-hot encoded prediction and ground truth vectors;
* `model`: logistic regression algorithm;
* `accuracy`: calculates the accuracy between a set of predictions and corresponding ground truth labels.

## 1. Introduction

In [2]:
### Importing required modules

In [3]:
import numpy as np
import os

In [4]:
### Setting environment variables

In [5]:
ENV_COLAB = False                # True if running in Google Colab instance

In [6]:
# Root directory
DIR_BASE = '' if not ENV_COLAB else '/content/'

In [7]:
# Subdirectory to save output files
DIR_OUT = os.path.join(DIR_BASE, 'out/')
# Subdirectory pointing to input data
DIR_SRC = os.path.join(DIR_BASE, 'data/')

### 1.1. Softmax

The [softmax function](https://en.wikipedia.org/wiki/Softmax_function) is a generalisation of the sigmoid [logistic function](https://en.wikipedia.org/wiki/Logistic_function) to multiple dimensions. In machine learning, particularly for logistic regression, the softmax function $\phi$ acts as a decision boundary applied to multi-class datasets, computing the probability of each observation $x_{i}$ belonging to one of $j = i,...,k$ class labels (assuming an independent relationship between the classes),

$$
\begin{align}
    P\left(y=j \ \vert \ z_{i}\right) = \phi_{softmax}\left(z_{i}\right) 
    = \frac{\mathcal{e}^{z_{i}}}{\sum_{j=0}^{k}\mathcal{e}^{z_{k}^{i}}}.
    \end{align}
$$
The input $z$ is defined to be
$$
\begin{align}
    z &= w_{0}x_{0} + w_{1}x_{1} + \ldots + w_{m}x_{m} = \sum_{i=0}^{m} w_{i}x_{i} = \mathrm{w}^{\top}\mathrm{x}.
    \end{align}
$$
such that $\mathrm{w}$ is the weight vector, $\mathrm{x}$ is the feature vector belonging to a single training observation, and $w_{0}$ is the bias unit.

The softmax function computes the probability for each class $P\left(y=j \vert x_{i}; w_{j}\right)$, then a correction step is applied to the predictions during training using a cost function that minimises the cross-entropy over the training set observations.

## 2. Programming Task

### 2.1. Softmax

In this exercise, you have to implement 4 different functions:

* `softmax`: compute the softmax of a vector. This function takes as input a tensor and outputs a discrete probability distribution.

In [8]:
### From Udacity's `logistic.py`

In [9]:
def softmax(logits):
    """
    softmax implementation
    args:
    - logits [tensor]: 1xN logits tensor
    returns:
    - soft_logits [tensor]: softmax of logits
    """
    # IMPLEMENT THIS FUNCTION
    
    soft_logits = np.exp(logits) / np.sum(np.exp(logits))
    return soft_logits

In [10]:
### Testing the softmax function
a = [1.0, 2.0, 3.0, 4.0, 1.0, 2.0, 3.0]
softmax(a)

array([0.02364054, 0.06426166, 0.1746813 , 0.474833  , 0.02364054,
       0.06426166, 0.1746813 ])

### 2.2. Cross-entropy

* `cross_entropy`: calculate the cross entropy loss given a vector of predictions (after softmax) and a vector of ground truth (one-hot vector).

In [11]:
### From Udacity's `logistic.py`

In [12]:
def cross_entropy(scaled_logits, one_hot):
    """
    Cross entropy loss implementation
    args:
    - scaled_logits [tensor]: NxC tensor where N batch size / C number of classes
    - one_hot [tensor]: one hot tensor
    returns:
    - loss [tensor]: cross entropy 
    """
    # IMPLEMENT THIS FUNCTION
    return nll

### 2.3. Logistic Regression model

* `model`: takes a batch of images (stack of images along the first dimensions) and feeds it through the logistic regression model.

In [13]:
### From Udacity's `logistic.py`

In [14]:
def model(X, W, b):
    """
    logistic regression model
    args:
    - X [tensor]: input HxWx3
    - W [tensor]: weights
    - b [tensor]: bias
    returns:
    - output [tensor]
    """
    # IMPLEMENT THIS FUNCTION
    return 

### 2.4. Prediction accuracy

* `accuracy`: given a vector of predictions and a vector of ground truth, calculate the accuracy.

In [15]:
### From Udacity's `logistic.py`

In [16]:
def accuracy(y_hat, Y):
    """
    calculate accuracy
    args:
    - y_hat [tensor]: NxC tensor of models predictions
    - y [tensor]: N tensor of ground truth classes
    returns:
    - acc [tensor]: accuracy
    """
    # IMPLEMENT THIS FUNCTION
    return acc

## Tips

You can leverage the `tf.boolean_mask` function to calculate the cross entropy. Keep in mind
that most elements of the ground truth vector are zeros.

## Credits
This assignment was prepared by Thomas Hossler and Michael Virgo et al., Winter 2021 (link [here](https://www.udacity.com/course/self-driving-car-engineer-nanodegree--nd0013)).

References


Helpful resources:
* [Softmax Regression and How is it Related to Logistic Regression? | KDnuggets](https://www.kdnuggets.com/2016/07/softmax-regression-related-logistic-regression.html)